In [96]:
#
##
### COMBINED NEURAL NETWORK

import numpy as np
import pandas as pd
from typing import List
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

class MultiOutputTransformer(BaseEstimator, TransformerMixin):

    def fit(self, y):
        if isinstance(y, pd.DataFrame):
            y = y.values
        y_class, y_reg = y[:, 0].reshape(-1,1), y[:, 1].reshape(-1,1)

        self.class_encoder_ = OneHotEncoder(sparse=False)
        self.reg_transformer_ = PowerTransformer()
        # Fit them to the input data
        self.class_encoder_.fit(y_class)
        self.reg_transformer_.fit(y_reg)
        # Save the number of classes
        self.n_classes_ = len(self.class_encoder_.categories_)
        self.n_outputs_expected_ = 2
        return self

    def transform(self, y):
        if isinstance(y, pd.DataFrame):
            y = y.values
        y_class, y_reg = y[:, 0].reshape(-1,1), y[:, 1].reshape(-1,1)
        # Apply transformers to input array
        y_class = self.class_encoder_.transform(y_class)
        y_reg = self.reg_transformer_.transform(y_reg)
        # Split the data into a list
        return [y_class, y_reg]

    def inverse_transform(self, y, return_proba=False):
        y_pred_reg = y[1]
        if return_proba:
            return y[0]
        else:
            y_pred_class = np.zeros_like(y[0])
            y_pred_class[np.arange(len(y[0])), np.argmax(y[0], axis=1)] = 1
            y_pred_class = self.class_encoder_.inverse_transform(y_pred_class)
        y_pred_reg = self.reg_transformer_.inverse_transform(y_pred_reg)
        return np.column_stack([y_pred_class, y_pred_reg])

    def get_metadata(self):
        return {
            "n_classes_": self.n_classes_,
            "n_outputs_expected_": self.n_outputs_expected_,
        }

from scikeras.wrappers import BaseWrapper
from tensorflow.keras.initializers import HeNormal, LecunNormal, HeNormal
from tensorflow.keras.layers import Input, Dense, BatchNormalization, concatenate, LeakyReLU
from tensorflow.keras import Model

class CombiNet(BaseWrapper):

    def __init__(self, activation = "selu",
        se_layers=1, se_units=256,
        re_layers=5, re_units=100,
        ce_layers=5, ce_units=100, cc_units=75,
        epochs=10, verbose=0,
        optimizer="adam", optimizer__clipvalue=1.0, **kwargs):
            super().__init__(**kwargs)
            self.activation = activation
            self.se_layers = se_layers
            self.se_units = se_units
            self.re_layers = re_layers
            self.re_units = re_units
            self.ce_layers = ce_layers
            self.ce_units = ce_units
            self.cc_units = cc_units
            self.epochs = epochs
            self.verbose = verbose
            self.__prediction_scope = {"classification":0,"regression":1,"full":range(2)}

    def _get_weight_init(self):
        if isinstance(self.activation, LeakyReLU):
            
            init = HeNormal()
        elif self.activation in ["selu", "elu"]:
            init = LecunNormal()
        else:
            init = HeNormal()  
        return init

    def _keras_build_fn(self, compile_kwargs):
        weight_init = self._get_weight_init()

        # shared extraction
        inp = Input(shape=(self.n_features_in_))
        fe = inp
        for i in range(self.se_layers):
            fe = Dense(self.se_units, self.activation,
                kernel_initializer=weight_init)(fe)
            fe = BatchNormalization()(fe)
        # regression branch
        re = fe
        for i in range(self.re_layers):
            re = Dense(self.re_units, self.activation,
                kernel_initializer=weight_init)(re)
            re = BatchNormalization()(re)
        rr_head = Dense(1,"linear")(re)
        # classification branch
        ce = fe
        for i in range(self.ce_layers):
            ce = Dense(self.ce_units, self.activation,
                kernel_initializer=weight_init)(ce)
            ce = BatchNormalization()(ce)
        cc = Dense(self.cc_units, self.activation,
            kernel_initializer=weight_init)(concatenate([ce, re]))
        cc = BatchNormalization()(cc)
        cc_head = Dense(2, "softmax")(cc)

        model = Model(inputs=inp, outputs=[cc_head, rr_head])
        model.compile(loss=["categorical_crossentropy","mse"], loss_weights=[.5,.5],
            optimizer=compile_kwargs["optimizer"])
        return model
        
    @property
    def target_encoder(self):
        return MultiOutputTransformer()
        
    def predict_proba(self, X):
        X = self.feature_encoder_.transform(X)
        y_pred = self.model_.predict(X)
        return self.target_encoder_.inverse_transform(y_pred, return_proba=True)

    def predict(self, X, scope="classification"):
        X = self.feature_encoder_.transform(X)
        y_pred = self.model_.predict(X)
        y_pred = self.target_encoder_.inverse_transform(y_pred)
        return y_pred[:,self.__prediction_scope[scope]]

In [ ]:
from sklearn.base import BaseEstimator, clone
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import  _SigmoidCalibration
from sklearn.isotonic import IsotonicRegression

class CombiNetCalibrationCV(BaseEstimator):
    def __init__(self, base_estimator, method="isotonic", cv=2):
        self.base_estimator = base_estimator
        self.method = method
        self.cv = cv

    def fit(self, X, y):
        calibrated_pairs = []
        base_estimator = clone(self.base_estimator)
        scv = StratifiedKFold(n_splits=2)
        for train_index, test_index in scv.split(X, y[:,0]):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # fit combinet
            base_estimator.fit(X_train, y_train)
            y_pred = base_estimator.predict_proba(X_test)
        
            # fit calibrator
            if self.method=="isotonic":
                calibrator = IsotonicRegression(y_min=0,y_max=1, out_of_bounds="clip")
                calibrator.fit(y_pred[:,1].T, y_test[:,0])
            if self.method=="sigmoid":
                calibrator = _SigmoidCalibration()
                calibrator.fit(y_pred[:,1].T, y_test[:,0])
            calibrated_pairs.append((base_estimator, calibrator))
        self.calibrated_pairs = calibrated_pairs
        return self

    def predict_proba(self, X):
        # calibrated positive class
        calibrated_class = np.zeros(shape=(X.shape[0], len(self.calibrated_pairs)))
        for i, calibrated_pair in enumerate(self.calibrated_pairs):
            raw_prediction = calibrated_pair[0].predict_proba(X)[:,1]
            calibrated_class[:,i] = calibrated_pair[1].predict(raw_prediction)
        calibrated_class = np.mean(calibrated_class, axis=1)
        return np.column_stack([1-calibrated_class, calibrated_class])

    def predict_reg(self, X):
        calibrated_reg = np.zeros(shape=(X.shape[0], len(self.calibrated_pairs)))
        for i, calibrated_pair in enumerate(self.calibrated_pairs):
            calibrated_reg[:,i] = calibrated_pair[0].predict(X, scope="regression")
        return np.mean(calibrated_reg, axis=1)
    
    def predict_full(self, X):
        return np.column_stack([(self.predict_proba(X)[:,1]>0.5).astype("int"),
            self.predict_reg(X)])
    
    def predict(self, X, scope="classification"):

        if scope=="classification":
           return (self.predict_proba(X)[:,1]>0.5).astype("int")
        if scope=="regression":
            return self.predict_reg(X)
        if scope=="full":
            return self.predict_full(X)

In [97]:
# data load
data = pd.read_parquet("../data/customer_model/retailrocket/")
train = data[data.week_step>2]
test = data[data.week_step==2]

out_cols = ["user_id", "target_event", "target_revenue", "week_step",
    "target_cap"]
feat_cols = [c for c in train.columns if c not in set(out_cols)]
target_cols = ["target_event", "target_cap"]

X = train.loc[:,feat_cols].values
y = train.loc[:,target_cols].values

calibrated_pipe =  CombiNetCalibrationCV(CombiNet()).fit(X, y)
calibrated_pipe.predict(X)